https://kosis.kr/statHtml/statHtml.do?orgId=381&tblId=DT_920005_B008&vw_cd=MT_ZTITLE&list_id=M2_25&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/kosis-airline.csv", encoding="cp949", dtype={"시점":"string"})
df.head(2)

,항공사별(1),시점,도착출발별(1),운항 (편),여객 (명),화물 (톤)
0,아시아나항공,2005.01,도착,6613,791439,28012
1,아시아나항공,2005.01,출발,6610,799835,29576


In [3]:
df.columns

Index(['항공사별(1)', '시점', '도착출발별(1)', '운항 (편)', '여객 (명)', '화물 (톤)'], dtype='object')

In [4]:
df.columns = ['항공사', '시점', '도착출발', '운항_편', '여객_명', '화물_톤']
df.head(1)

,항공사,시점,도착출발,운항_편,여객_명,화물_톤
0,아시아나항공,2005.01,도착,6613,791439,28012


In [5]:
df["시점"].unique()

<StringArray>
['2005.01', '2005.02', '2005.03', '2005.04', '2005.05', '2005.06', '2005.07',
 '2005.08', '2005.09', '2005.10',
 ...
 '2022.11', '2022.12', '2023.01', '2023.02', '2023.03', '2023.04', '2023.05',
 '2023.06', '2023.07', '2023.08']
Length: 224, dtype: string

In [6]:
df["연도"] = df["시점"].str[:4]
df["월"] = df["시점"].str.split(".", expand=True)[1].astype(int)

In [7]:
df["항공사"].value_counts()

항공사
아시아나항공     448
대한항공       448
외항사        448
제주항공       414
진에어        364
에어부산       358
티웨이항공      312
이스타항공      284
에어인천       252
에어서울       172
플라이강원       86
에어로케이항공     62
에어프레미아      48
Name: count, dtype: int64

In [8]:
pd.crosstab(df["항공사"], df["연도"])

연도,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
항공사,,,,,,,,,,,,,,,,,,,
대한항공,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,16
아시아나항공,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,16
에어로케이항공,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,24,16
에어부산,0,0,0,6,24,24,24,24,24,24,24,24,24,24,24,24,24,24,16
에어서울,0,0,0,0,0,0,0,0,0,0,0,12,24,24,24,24,24,24,16
에어인천,0,0,0,0,0,0,0,0,20,24,24,24,24,24,24,24,24,24,16
에어프레미아,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,24,16
외항사,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,16
이스타항공,0,0,0,0,24,24,24,24,24,24,24,24,24,24,24,8,0,0,12


In [9]:
year_company = pd.crosstab(df["연도"], df["항공사"], values=df["여객_명"], aggfunc="sum").fillna(0)
year_company.style.background_gradient(cmap="Blues", axis=None).format("{:,.0f}")

항공사,대한항공,아시아나항공,에어로케이항공,에어부산,에어서울,에어인천,에어프레미아,외항사,이스타항공,제주항공,진에어,티웨이항공,플라이강원
연도,,,,,,,,,,,,,
2005,"34,721,098","18,535,331",0,0,0,0,0,"11,319,209",0,0,0,0,0
2006,"34,990,013","19,284,957",0,0,0,0,0,"12,856,097",0,"508,612",0,0,0
2007,"34,654,664","19,352,387",0,0,0,0,0,"15,483,064",0,"1,649,508",0,0,0
2008,"33,096,441","19,786,251",0,"205,580",0,0,0,"14,579,456",0,"1,955,455","329,510",0,0
2009,"30,397,856","18,103,278",0,"2,841,444",0,0,0,"11,579,865","1,996,549","2,902,584","2,431,545",0,0
2010,"33,095,526","19,470,441",0,"4,134,712",0,0,0,"13,495,430","3,726,008","3,960,525","3,009,247","308,876",0
2011,"31,644,789","19,692,263",0,"4,632,692",0,0,0,"14,590,031","3,366,115","5,365,058","3,096,266","2,999,202",0
2012,"32,412,913","20,541,440",0,"4,955,130",0,0,0,"15,982,369","3,918,272","6,551,702","3,893,786","3,498,629",0
2013,"30,837,092","21,158,822",0,"5,855,989",0,0,0,"17,819,459","4,639,327","7,567,220","4,655,414","4,106,794",0
